In [1]:
from itertools import cycle
import numpy as np

In [2]:
with open("day_17_input.txt") as file:
    inp = file.readline().rstrip()
    jets_n = len(inp)
    jets = cycle(inp)

In [3]:
jets_n  # 10091 is a prime number!

10091

In [4]:
rocks = [
    np.array([[1, 1, 1, 1]]),
    np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]]),
    np.array([[1, 1, 1], [0, 0, 1], [0, 0, 1]]),
    np.array([[1], [1], [1], [1]]),
    np.array([[1, 1], [1, 1]]),
]
rock_gen = cycle(rocks)

#### Task 1

In [5]:
# Takeing the upper bound on how tall the tower could be:
task_1_grid = np.zeros((((2022 // 5) + 1) * 13, 7))

# Adding stone walls to facilitate coding:
task_1_grid = np.append(np.ones((1, 7)) * 9, task_1_grid, axis=0)
task_1_grid = np.append(np.ones((task_1_grid.shape[0], 1)) * 9, task_1_grid, axis=1)
task_1_grid = np.append(task_1_grid, np.ones((task_1_grid.shape[0], 1)) * 9, axis=1)

In [6]:
# For a more robust and verbose implementation look at Task 2:
cur_height = 0
rock_num = 0
moves = 0
while rock_num < 2022:
    rock = next(rock_gen)
    rock_pos = np.array([cur_height + 4, 3])
    while True:
        moves += 1
        jet = next(jets)
        shift = -1 + 2 * (jet == ">")
        cur_y, cur_x = rock_pos
        if (
            np.max(
                task_1_grid[
                    cur_y : (cur_y + rock.shape[0]),
                    (cur_x + shift) : (cur_x + shift + rock.shape[1]),
                ]
                + rock
            )
            == 1
        ):
            rock_pos += [0, shift]
            cur_y, cur_x = rock_pos
        if (
            np.max(
                task_1_grid[
                    (cur_y - 1) : (cur_y - 1 + rock.shape[0]),
                    (cur_x) : (cur_x + rock.shape[1]),
                ]
                + rock
            )
            == 1
        ):
            rock_pos += [-1, 0]
            cur_y, cur_x = rock_pos
        else:
            task_1_grid[
                (cur_y) : (cur_y + rock.shape[0]), (cur_x) : (cur_x + rock.shape[1])
            ] += rock
            cur_height = max(cur_height, cur_y + rock.shape[0] - 1)
            break
    rock_num += 1

In [7]:
# Task 1 solution:
cur_height

3224

#### Task 2

In [8]:
# Reset the cycles!!!
jets = cycle(inp)
rock_gen = cycle(rocks)

In [9]:
# Takeing the upper bound on how tall the tower could be:
task_2_grid = np.zeros((7, 7))

# Adding  walls to facilitate coding:
task_2_grid = np.append(np.ones((1, 7)), task_2_grid, axis=0)
task_2_grid = np.append(np.ones((task_2_grid.shape[0], 1)), task_2_grid, axis=1)
task_2_grid = np.append(task_2_grid, np.ones((task_2_grid.shape[0], 1)), axis=1)

# Needed to track and trim the tower
not_reachable = np.copy(task_2_grid)

In [10]:
# Updated version from 1 with trimming
cur_height = 0
height_trimmed = 0
rock_num = 0
moves = 0
grid_dict = dict()
# While unencountered configuration of reachability, rocks and jets:
while (
    tuple(tuple(row) for row in not_reachable),
    rock_num % 5,
    moves % jets_n,
) not in grid_dict:
    rock = next(rock_gen)
    rock_pos = np.array([cur_height + 4, 3])
    # Record the reachability, rock and jet configoration:
    grid_dict[
        (tuple(tuple(row) for row in not_reachable), rock_num % 5, moves % jets_n)
    ] = (rock_num, height_trimmed + cur_height)
    while True:
        moves += 1
        jet = next(jets)
        shift = -1 + 2 * (jet == ">")
        cur_y, cur_x = rock_pos
        if (
            np.max(
                task_2_grid[
                    cur_y : (cur_y + rock.shape[0]),
                    (cur_x + shift) : (cur_x + shift + rock.shape[1]),
                ]
                + rock
            )
            == 1
        ):
            # If there is no collision, move the rock with the jet:
            rock_pos += [0, shift]
            cur_y, cur_x = rock_pos
        if (
            np.max(
                task_2_grid[
                    (cur_y - 1) : (cur_y - 1 + rock.shape[0]),
                    (cur_x) : (cur_x + rock.shape[1]),
                ]
                + rock
            )
            == 1
        ):
            # If there is no collision, move the rock down:
            rock_pos += [-1, 0]
            cur_y, cur_x = rock_pos
        else:
            # If there is a collision, record the rock:
            task_2_grid[
                (cur_y) : (cur_y + rock.shape[0]), (cur_x) : (cur_x + rock.shape[1])
            ] += rock
            not_reachable[
                (cur_y) : (cur_y + rock.shape[0]), (cur_x) : (cur_x + rock.shape[1])
            ] += rock
            # Update non-reachablity:
            for y in range(cur_height + 4, 0, -1):
                for x in range(1, 8):
                    if not not_reachable[y, x]:
                        # Is it reachable from top?
                        not_reachable[y, x] = not_reachable[y + 1, x]
                for x in range(1, 8):
                    if not task_2_grid[y, x]:
                        # Is it reachable from left or right?
                        not_reachable[y, x] = min(
                            not_reachable[y, x],
                            not_reachable[y, x - 1],
                            not_reachable[y, x + 1],
                        )
            # What's the total height of the lowest reachable cell?
            col_mins = (not_reachable[: (cur_height + 4), 1:8] == 0).argmax(axis=0)
            to_trim = 0
            # Trim the grid if possible:
            if col_mins.min() > 1:
                to_trim = col_mins.min() - 1
                task_2_grid = task_2_grid[to_trim:, :]
                not_reachable = not_reachable[to_trim:, :]
                height_trimmed += to_trim
            cur_height = max(cur_height - to_trim, cur_y + rock.shape[0] - 1 - to_trim)
            # Extend the grid if necessary:
            if cur_height + 8 > task_2_grid.shape[0]:
                task_2_grid = np.append(
                    task_2_grid,
                    np.concatenate(
                        [
                            np.ones((cur_height - task_2_grid.shape[0] + 8, 1)),
                            np.zeros((cur_height - task_2_grid.shape[0] + 8, 7)),
                            np.ones((cur_height - task_2_grid.shape[0] + 8, 1)),
                        ],
                        axis=1,
                    ),
                    axis=0,
                )
                not_reachable = np.append(
                    not_reachable,
                    np.concatenate(
                        [
                            np.ones((cur_height - not_reachable.shape[0] + 8, 1)),
                            np.zeros((cur_height - not_reachable.shape[0] + 8, 7)),
                            np.ones((cur_height - not_reachable.shape[0] + 8, 1)),
                        ],
                        axis=1,
                    ),
                    axis=0,
                )
            break
    rock_num += 1

In [11]:
# The grid when the cycle is detected:
print(np.flip(task_2_grid, axis=0))

[[1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 0. 0. 0. 1.]
 [1. 0. 1. 1. 0. 0. 0. 0. 1.]
 [1. 0. 1. 1. 0. 0. 0. 0. 1.]
 [1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [1. 0. 0. 1. 0. 1. 0. 0. 1.]
 [1. 0. 0. 1. 0. 1. 0. 0. 1.]
 [1. 0. 0. 1. 1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 1. 1. 1. 0. 1.]
 [1. 0. 0. 1. 1. 1. 0. 0. 1.]
 [1. 0. 0. 1. 1. 1. 0. 0. 1.]
 [1. 0. 0. 1. 1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 1. 0. 0. 0. 1.]
 [1. 0. 1. 1. 1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 1. 1. 1. 0. 1.]
 [1. 0. 0. 0. 0. 1. 0. 0. 1.]
 [1. 0. 0. 1. 1. 1. 1. 0. 1.]
 [1. 0. 0. 0. 0. 0. 1. 0. 1.]
 [1. 0. 0. 0. 0. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 1. 1. 1. 1.]]


In [12]:
# The non-reachable cells when the cycle is detected:
print(np.flip(not_reachable, axis=0))

[[1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 0. 0. 0. 1.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [13]:
pre_cycle_rocks, pre_cycle_height = grid_dict[
    (tuple(tuple(row) for row in not_reachable), rock_num % 5, moves % jets_n)
]
pre_cycle_rocks, pre_cycle_height

(176, 261)

In [14]:
post_cycle_rocks, post_cycle_height = rock_num, height_trimmed + cur_height
post_cycle_rocks, post_cycle_height

(1921, 3046)

In [15]:
cycle_len = post_cycle_rocks - pre_cycle_rocks
cycle_len

1745

In [16]:
cycle_height = post_cycle_height - pre_cycle_height
cycle_height

2785

In [17]:
base_rocks, cycle_iterations = 1000000000000 % cycle_len, 1000000000000 // cycle_len
base_rocks, cycle_iterations

(1010, 573065902)

In [18]:
# _ would be assigned the number of base_rocks
_, base_height = [
    grid_dict[key] for key in grid_dict if grid_dict[key][0] == base_rocks
][0]
_, base_height

(1010, 1621)

In [19]:
# Task 2 result:
base_height + cycle_iterations * cycle_height  # CORRECTO!!!

1595988538691